<a href="https://colab.research.google.com/github/Omarkouta73/RAG-Applications-In-Web-Data-Extraction/blob/main/RAGTextArabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install faiss-cpu
!pip install selenium
!pip install google-generativeai
!pip install -qU langchain-text-splitters
#!pip install langchain_experimental langchain_openai
#!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 17.2 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
import openai
import faiss
import numpy as np
import torch
from transformers import pipeline
import google.generativeai as genai
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_text_splitters import HTMLHeaderTextSplitter
from langchain_text_splitters import HTMLSectionSplitter
#from langchain_experimental.text_splitter import SemanticChunker
#from langchain_openai.embeddings import OpenAIEmbeddings
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# Step 1: Fetch HTML content
def get_cleaned_text(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Remove the specific div with id="customer-reviews_feature_div"
    customer_reviews_div = soup.find('div', id='customer-reviews_feature_div')
    if customer_reviews_div and hasattr(customer_reviews_div, 'decompose'):
        customer_reviews_div.decompose()

    # Remove unnecessary tags (scripts, styles, iframes, etc.)
    for tag in soup.find_all(['style', 'iframe', 'footer', 'header']):
        if tag and hasattr(tag, 'decompose'):
            tag.decompose()

    # Remove tags with ad-related keywords
    ad_keywords = ['ad', 'banner', 'promo', 'footer', 'sponsor', 'select', 'button']
    for tag in soup.find_all(ad_keywords):
        if tag and hasattr(tag, 'decompose'):
            tag.decompose()

    # Remove empty tags
    for tag in soup.find_all():
        if tag and not tag.get_text(strip=True):
            if hasattr(tag, 'decompose'):
                tag.decompose()

    return soup

def fetch_html(url):
    # List of user agents
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'
    ]

    # Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(f"user-agent={random.choice(user_agents)}")

    # Set location to Egypt
    chrome_options.add_argument("--lang=ar-EG")
    chrome_options.add_experimental_option("prefs", {
        "intl.accept_languages": "ar-EG,ar",
        "profile.default_content_setting_values.geolocation": 1
    })

    # Add geolocation for Cairo, Egypt
    chrome_options.add_argument("--enable-geolocation")
    chrome_options.add_argument("--use-fake-ui-for-media-stream")
    chrome_options.add_argument("--use-fake-device-for-media-stream")
    chrome_options.add_experimental_option("prefs", {
        "profile.default_content_settings.geolocation": 1,
        "profile.managed_default_content_settings.geolocation": 1,
        "profile.default_content_setting_values.geolocation": 1
    })

    service = Service()  # Windows
    driver = webdriver.Chrome(service=service, options=chrome_options)

    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", {
        "latitude": 30.0444,
        "longitude": 31.2357,
        "accuracy": 100
    })

    time.sleep(random.uniform(5, 10))

    driver.get(url)

    return get_cleaned_text(driver.page_source)

def read_html(dir):
  with open(dir, "r", encoding="utf-8") as file:
        return file.read()

In [ ]:
def read_md(dir):
  try:
      with open(dir, "r", encoding="utf-8") as file:
          content = file.read()
      return content
  except FileNotFoundError:
      print(f"Error: The file '{dir}' was not found.")
      return None
  except Exception as e:
      print(f"An error occurred while reading the file: {e}")
      return None

In [ ]:
# Step 2: Text Chunking using RCTS
def recursive_text_split(text, delimiters, max_chunk_size):
    chunks = []
    for delimiter in delimiters:
        parts = text.split(delimiter)
        for part in parts:
            if len(part) <= max_chunk_size:
                chunks.append(part)
            else:
                chunks.extend(recursive_text_split(part, delimiters[1:], max_chunk_size))
    return chunks

def langchain_RCTS(text, max_chunk_size):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=max_chunk_size,
      chunk_overlap=0,
      length_function=len,
  )
  return text_splitter.split_text(text)

def langchain_SC(text):
  text_splitter = SemanticChunker(
      OpenAIEmbeddings(), breakpoint_threshold_type="standard_deviation"
  )
  return text_splitter.split_text(text)

def langchain_MHTS(text):
  # Take screenshot of the webpage
  # Convert the image to pdf
  # convert pdf to markdown
  # Split on markdown
  headers_to_split_on = [
      ("#", "Header 1"),
      ("##", "Header 2"),
      ("###", "Header 3"),
      ("####", "Header 4"),
  ]
  markdown_splitter = MarkdownHeaderTextSplitter(
      headers_to_split_on=headers_to_split_on,
  )
  return markdown_splitter.split_text(text)

def langchain_HSS(text):
  headers_to_split_on = [
      ("h1", "Header 1"),
      ("h2", "Header 2"),
      ("h3", "Header 3"),
      ("h4", "Header 4"),
  ]

  html_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)

  html_header_splits = html_splitter.split_text(text)

  chunk_size = 500
  chunk_overlap = 0
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size, chunk_overlap=chunk_overlap
  )

  # Split
  return text_splitter.split_documents(html_header_splits)

def langchain_HHTS(text):
  headers_to_split_on = [
      ("h1", "Header 1"),
      ("h2", "Header 2"),
      ("h3", "Header 3"),
  ]

  html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
  return html_splitter.split_text(text)

In [ ]:
# Step 3: Text Embedding
def embed_text(text, model):
    # Move model to GPU
    #model.to(device)

    # Get token embeddings
    token_embeddings = model(text, return_tensors="pt")[0].to(device)  # Move embeddings to GPU
    # Average the token embeddings to get a single vector
    chunk_embedding = token_embeddings.mean(dim=0).detach().cpu().numpy()  # Move back to CPU for FAISS
    return chunk_embedding

In [ ]:
# Step 4: Query Embedding and Similarity Search
def search_similar_chunks(query, chunks, embeddings, model, top_k=3):
    query_embedding = embed_text(query, model)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    distances, indices = index.search(np.array([query_embedding]), top_k)
    return [chunks[i] for i in indices[0]]

In [ ]:
# Step 5: LLM Processing with Gemini
def extract_info_with_llm(chunks, prompt):
    # Combine chunks into a single string
    combined_chunks = " ".join(chunks)

    # Initialize Gemini
    genai.configure(api_key="YOUR_GEMINI_API_KEY")  # Replace with your Gemini API key

    # Use the Gemini model
    model = genai.GenerativeModel('gemini-2.0-flash')  # Use the Gemini Pro model

    # Generate a response
    response = model.generate_content(f"""using these information: {combined_chunks} make your best to {prompt}
                                      , and provide it in ```json ``` format.
                                      give me 'NAN' if there is no direct answer in the provided information.""",
                                      generation_config={"max_output_tokens": 500})

    # Return the generated text
    return response.text

In [ ]:
# Example Usage
url = "https://www.aliexpress.com/item/1005006251922668.html?spm=a2g0o.tm1000010823.d0.1.468f11b578s8Ik&sourceType=1&pvid=251a342c-c69f-41bf-9b87-f63193396869&pdp_ext_f=%7B%22ship_from%22%3A%22CN%22%2C%22sku_id%22%3A%2212000036474960600%22%7D&scm=1007.51228.386262.0&scm-url=1007.51228.386262.0&scm_id=1007.51228.386262.0&channel=bigSave&aecmd=true&_gl=1*1ybfags*_gcl_au*MTgxMDU3MDYzOC4xNzQxODMwMzY5*_ga*ODM2MDQ1OTQuMTc0MTgzMDM2OQ..*_ga_VED1YSGNC7*MTc0MjEwMTkzMS4yLjEuMTc0MjEwMTkzNC41Ny4wLjA."
html = fetch_html(url)

In [ ]:
#html = read_html("/content/page.html")

In [ ]:
techniques = {
    #"Text Splitter": langchain_RCTS(text=str(html), max_chunk_size=500),
    #"Markdown Splitter": langchain_MHTS(text=read_md("/content/html-to-markdown.md.txt")),
    "HTML Text Splitter": langchain_HHTS(str(html)),
    #"HTML Selector": langchain_HSS(str(html)),
}

In [ ]:
# # Embed chunks
# model1 = pipeline('feature-extraction', model='distilbert-base-uncased', padding=True, truncation=True, add_special_tokens = True)

# # Option 1: Better pre-trained model
# model2 = pipeline('feature-extraction',
#                  model='sentence-transformers/all-MiniLM-L6-v2',
#                  padding=True,
#                  truncation=True,
#                  add_special_tokens=True)

# Option 2: Another good alternative
model3 = pipeline('feature-extraction',
                 model='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
                 padding=True,
                 truncation=True,
                 add_special_tokens=True)

Device set to use cpu
Device set to use cpu


config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


# Manual

In [ ]:
chunks = techniques['Text Splitter']; chunks #ch_con = [chunk.page_content for chunk in chunks];

[Document(metadata={}, page_content='Xbox Core Wireless Gaming Controller – Deep Pink Series X|S, One, Windows PC, Android, and iOS\n==============================================================================================  \nXbox Core Wireless Gaming Controller – Deep Pink Series X|S, One, Windows PC, Android, and iOS Platform : _4.6 out of 5 stars_ 81,522 ratings $ $52.99 $38.56 Shipping & Import Fees Deposit to Egypt Delivery Sunday, February 9. Order within 6 hrs 47 mins Or fastest delivery Thursday, February 6 52. 99 . 6 hrs 47 mins  \n$38.56 Shipping & Import Fees Deposit to Egypt'),
 Document(metadata={'Header 2': 'Shipping & Fee Details'}, page_content='Price  \n$52.99  \nAmazonGlobal Shipping  \n$19.23  \n$19.33  \nTotal  \n$91.55')]

In [ ]:
embeddings = np.array([embed_text(chunk, model) for chunk in chunks])

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
# Query
query = "Extract product price"
similar_chunks = search_similar_chunks(query, chunks, embeddings, top_k=3)

In [ ]:
similar_chunks

['next word to use in a text message, but does not have the power to generate coherent',
 'example below, the vectors corresponding to the words “Write”, “a”, “story”, and “.” become the modified vectors that carry information about their position, labeled “Write (1)”, “a (2)”, “story (3)”, and “.',
 'architecture of a transformer']

# Automatically

In [ ]:
def prompt(query, model, k=3):
  prompt = query
  start_time = time.time()
  no_chunk_result = extract_info_with_llm(str(html), prompt)
  no_chunk_time = (time.time() - start_time) * 1000  # Convert to milliseconds
  print(f"No-Chunking: {no_chunk_result}")
  print(f"Time: {no_chunk_time:.2f} ms")
  print("==============================================================================")
  # Measure each chunking technique
  for key, chunks in techniques.items():
      start_time = time.time()

      if type(chunks[0]) != str:
          chunks = [chunk.page_content for chunk in chunks]
      print(key,":")
      print("Chunks: ", chunks)

      embeddings = np.array([embed_text(chunk, model) for chunk in chunks])
      print("Embeddings Shape: ", embeddings.shape)

      similar_chunks = search_similar_chunks(query, chunks, embeddings, model, top_k=k)
      print("Similar Chunks: ", similar_chunks)

      result = extract_info_with_llm(similar_chunks, prompt)
      execution_time = (time.time() - start_time) * 1000  # Convert to milliseconds

      print("Response:", result)
      print(f"Time: {execution_time:.2f} ms")
      print("==============================================================================")

## paraphrase-multilingual-MiniLM-L12-v2

In [ ]:
prompt("Who is the product's manufacturer ?", model3, k=3)

No-Chunking: ```json
{
  "manufacturer": "ابل"
}
```
Time: 1589.91 ms
Text Splitter :
Chunks:  ['<html dir="rtl" lang="ar" style="--window-height: 388px;"><head>\n<title>ابل ايفون 16 بشريحة اتصال واحدة، 128 جيجابايت، 8 جيجا رام، شبكة الجيل الخامس - التراماين</title>\n</head>\n<body>', '<div id="__nuxt"><!-- --><div id="__layout"><div class="bg-primary-300" data-v-786ee1b8="" id="app" style="--header-height: 140px;">   <!-- --> <main class="bg-primary-300 z-auto relative pb-16" data-v-786ee1b8="" id="main"><div data-fetch-key="PageResolver:0" data-v-786ee1b8=""><div class="product-page" data-fetch-key="data-v-7a50c244:0" data-v-7a50c244=""> <div class="[ container mx-auto ] product-container mt-2" data-v-7a50c244=""><div class="flex lg:hidden flex-col order-1 [ px-6 lg:px-0 ]"', 'data-v-7a50c244=""> <div class="lg:order-2 flex items-center justify-between gap-2" data-v-7a50c244=""><h1 class="text-lg font-bold text-primary-700" data-v-7a50c244="">ابل ايفون 16 بشريحة اتصال واحدة، 128 جيجا

In [ ]:
prompt("What is the OS version ?", model3, k=3)

No-Chunking: ```json
{
  "OS Version": "iOS 18, upgradable to iOS 18.2"
}
```
Time: 1195.04 ms
Text Splitter :
Chunks:  ['<html dir="rtl" lang="ar" style="--window-height: 388px;"><head>\n<title>ابل ايفون 16 بشريحة اتصال واحدة، 128 جيجابايت، 8 جيجا رام، شبكة الجيل الخامس - التراماين</title>\n</head>\n<body>', '<div id="__nuxt"><!-- --><div id="__layout"><div class="bg-primary-300" data-v-786ee1b8="" id="app" style="--header-height: 140px;">   <!-- --> <main class="bg-primary-300 z-auto relative pb-16" data-v-786ee1b8="" id="main"><div data-fetch-key="PageResolver:0" data-v-786ee1b8=""><div class="product-page" data-fetch-key="data-v-7a50c244:0" data-v-7a50c244=""> <div class="[ container mx-auto ] product-container mt-2" data-v-7a50c244=""><div class="flex lg:hidden flex-col order-1 [ px-6 lg:px-0 ]"', 'data-v-7a50c244=""> <div class="lg:order-2 flex items-center justify-between gap-2" data-v-7a50c244=""><h1 class="text-lg font-bold text-primary-700" data-v-7a50c244="">ابل ايفون 16 بشر